# Getting Started with agent-workshop

Welcome to **agent-workshop**, a batteries-included framework for building automation-focused AI agents with full observability.

## What is agent-workshop?

agent-workshop provides:

- **Pre-built agents** for common tasks (validation, code review, releases)
- **Two agent patterns**: Simple agents (80% use case) and LangGraph workflows (15%)
- **Dual provider system**: Claude Code SDK (dev) or Anthropic API (prod)
- **Full observability**: Automatic Langfuse tracing for all operations
- **Blueprint system**: Define agents in YAML, generate Python code

## Installation

```bash
# Basic installation
pip install agent-workshop

# With pre-built agents
pip install agent-workshop[agents]

# With Claude Code SDK support (development)
pip install agent-workshop[claude-agent]

# Full installation
pip install agent-workshop[agents,claude-agent,dev]
```

## Provider Configuration

agent-workshop supports two LLM providers:

### Option 1: Claude Code SDK (Development)

Flat $20/month pricing - ideal for development and testing.

```bash
export AGENT_WORKSHOP_ENV=development
export CLAUDE_SDK_ENABLED=true
```

### Option 2: Anthropic API (Production)

Pay-per-token pricing - ideal for production use.

```bash
export AGENT_WORKSHOP_ENV=production
export ANTHROPIC_API_KEY=sk-ant-...
```

The framework auto-detects which provider to use based on environment.

In [1]:
# Check your current configuration
import os

print("Current Configuration:")
print("=" * 40)
print(f"AGENT_WORKSHOP_ENV: {os.getenv('AGENT_WORKSHOP_ENV', 'development')}")
print(f"CLAUDE_SDK_ENABLED: {os.getenv('CLAUDE_SDK_ENABLED', 'true')}")
print(f"ANTHROPIC_API_KEY: {'Set' if os.getenv('ANTHROPIC_API_KEY') else 'Not set'}")
print(f"LANGFUSE_ENABLED: {os.getenv('LANGFUSE_ENABLED', 'false')}")

Current Configuration:
AGENT_WORKSHOP_ENV: development
CLAUDE_SDK_ENABLED: true
ANTHROPIC_API_KEY: Set
LANGFUSE_ENABLED: true


## Agent Types

### 1. Simple Agents (80% of use cases)

Single input to single output pattern. Best for:
- Document validation
- Code review
- Content analysis

```python
from agent_workshop import Agent, Config

class MyAgent(Agent):
    async def run(self, content: str) -> dict:
        messages = [{"role": "user", "content": content}]
        result = await self.complete(messages)
        return {"result": result}
```

### 2. LangGraph Workflows (15% of use cases)

Multi-step workflows with state management. Best for:
- Complex validation pipelines
- Multi-stage review processes
- Workflows with conditional logic

```python
from agent_workshop.workflows import LangGraphAgent
from langgraph.graph import StateGraph, END

class MyWorkflow(LangGraphAgent):
    def build_graph(self):
        workflow = StateGraph(dict)
        workflow.add_node("step1", self.step1)
        workflow.add_node("step2", self.step2)
        workflow.add_edge("step1", "step2")
        workflow.add_edge("step2", END)
        workflow.set_entry_point("step1")
        return workflow.compile()
```

## Pre-Built Agents

agent-workshop includes production-ready agents:

| Agent | Type | Use Case |
|-------|------|----------|
| **DeliverableValidator** | Simple | Validate documents, reports, research papers |
| **ValidationPipeline** | LangGraph | Multi-step validation with quick scan + detail |
| **CodeReviewer** | Simple | Review code for quality and security |
| **PRPipeline** | LangGraph | Security scan, Quality review, Summary |
| **ReleasePipeline** | Hybrid | Automated releases with git/PR operations |
| **NotebookValidator** | Simple | Validate Jupyter notebooks |
| **AgentBuilder** | Meta | Generate agents from YAML blueprints |

In [ ]:
# Quick example: Using DeliverableValidator
from agent_workshop.agents.validators import DeliverableValidator
from agent_workshop import Config

config = Config()
validator = DeliverableValidator(config)

print(f"Validator created with provider: {validator.provider_name}")
print(f"Model: {validator.model_name}")

In [ ]:
# Quick example: Using CodeReviewer
from agent_workshop.agents.software_dev import CodeReviewer, get_preset
from agent_workshop import Config

config = Config()

# Using default settings
reviewer = CodeReviewer(config)

# Or using a preset
preset = get_preset("security_focused")
security_reviewer = CodeReviewer(config, **preset)

print("CodeReviewer presets available:")
from agent_workshop.agents.software_dev import list_presets
for p in list_presets():
    print(f"  - {p['name']}: {p['description']}")

## Running an Agent

All agents follow the same pattern:

In [ ]:
# Example: Run the CodeReviewer (commented out to avoid API calls)

sample_code = '''
def process_data(data):
    # This code has security issues that the reviewer will detect
    API_KEY = "sk-secret-12345"  # Hardcoded secret
    query = f"SELECT * FROM users WHERE id = {data}"  # SQL injection risk
    return query
'''

# Uncomment to run:
# result = await reviewer.run(sample_code)
# print(result)

print("Sample code to review:")
print(sample_code)

## Tutorial Notebooks

This is part of a series of tutorials:

1. **[01_deliverable_validator.ipynb](./01_deliverable_validator.ipynb)** - Document validation with presets
2. **[02_validation_pipeline.ipynb](./02_validation_pipeline.ipynb)** - Multi-step validation workflow
3. **[03_code_reviewer.ipynb](./03_code_reviewer.ipynb)** - Code review agent
4. **[04_pr_pipeline.ipynb](./04_pr_pipeline.ipynb)** - PR review workflow
5. **[05_release_pipeline.ipynb](./05_release_pipeline.ipynb)** - Automated release workflow
6. **[06_notebook_validator.ipynb](./06_notebook_validator.ipynb)** - Jupyter notebook validation
7. **[07_blueprint_system.ipynb](./07_blueprint_system.ipynb)** - Creating agents from YAML

## Next Steps

1. **Try a simple agent**: Start with [01_deliverable_validator.ipynb](./01_deliverable_validator.ipynb)
2. **Explore workflows**: See [02_validation_pipeline.ipynb](./02_validation_pipeline.ipynb)
3. **Automate releases**: Check [05_release_pipeline.ipynb](./05_release_pipeline.ipynb)
4. **Build custom agents**: Learn from [07_blueprint_system.ipynb](./07_blueprint_system.ipynb)

In [ ]:
import os
import sys

# Add src to path
sys.path.insert(0, "/home/trentleslie/Insync/projects/agent-workshop/src")

# Set environment
os.environ["AGENT_WORKSHOP_ENV"] = "development"
os.environ["CLAUDE_SDK_ENABLED"] = "true"
os.environ["LANGFUSE_ENABLED"] = "false"

from agent_workshop import Config
from agent_workshop.agents.software_dev import PRCommentProcessor

# Real comments from PR #1
PR_COMMENTS = [
    {
        "id": "2649346649",
        "body": """**logic:** Command injection risk: double-quotes with `{commit_message}` interpolation allows shell injection if LLM output contains special characters

The blueprint uses `"git commit -m \\"{commit_message}\\""` which is unsafe.

```suggestion
          command: "git commit -m '{commit_message}'"
```""",
        "path": "blueprints/specs/software_dev_release_pipeline.yaml",
        "line": 100,
        "addressed": False,
    },
    {
        "id": "2649346652", 
        "body": """**logic:** Same command injection risk with `{pr_body}` - multiline content with backticks or $() will execute

```suggestion
          command: "gh pr create --base {base_branch} --title 'Release v{version}' --body '{pr_body}'"
```""",
        "path": "blueprints/specs/software_dev_release_pipeline.yaml",
        "line": 127,
        "addressed": False,
    },
]

print("Initializing PRCommentProcessor...")
config = Config()
processor = PRCommentProcessor(
    config,
    max_iterations=5,
    working_dir="/home/trentleslie/Insync/projects/agent-workshop",
)
print("Processor initialized!")
print(f"Graph nodes: {list(processor.graph.nodes.keys())}")


In [ ]:
# Check if processor was created
print(f"Processor type: {type(processor)}")
print(f"Graph nodes: {list(processor.graph.nodes.keys())}")
print(f"Working dir: {processor._working_dir}")
print(f"Max iterations: {processor.max_iterations}")
